In [2]:
import pandas as pd
from openai import OpenAI
import time

# === 🔐 API-Setup für OpenRouter ===
client = OpenAI(
    api_key="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",  
    base_url="https://openrouter.ai/api/v1"
)

In [3]:
# === 📋 Modelle definieren ===
MODELS = {
    "gpt-4": "openai/gpt-4",
    "claude-3-opus": "anthropic/claude-3-opus",
    "llama-3-70b": "meta-llama/llama-3.3-70b-instruct",
    "mistral-7b": "mistralai/mistral-7b-instruct",
}

In [4]:
# === 🧠 Funktion: Prompt an LLM schicken ===
def run_prompt(model, prompt):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=512
    )
    return response.choices[0].message.content

In [5]:
# === 🧱 Produktbeschreibung aus Gruppe erzeugen ===
def create_product_description(group):
    lines = []
    for _, row in group.iterrows():
        line = f"{row['PropertyDefinition']}: {row['PropertyValue']}"
        if pd.notnull(row['UnitName']):
            line += f" {row['UnitName']}"
        lines.append(line)
    return "\n".join(lines)

In [6]:
# === 🧾 Prompt vorbereiten ===
def generate_prompt_from_group(group):
    product_text = create_product_description(group)
    return f"""
Bitte extrahiere die wichtigsten technischen Produktspezifikationen aus folgendem Text und gib sie als JSON mit sinnvollen Attributnamen zurück.

Produktdaten:
{product_text}
"""

In [7]:
# === 🚀 Main-Skript ===
def main():
    # 🔄 Datei laden
    df = pd.read_parquet("241212_processedproperties_fhnw.parquet")  
    grouped = df.groupby("ProductId")

    results = []

    for product_id, group in list(grouped)[:100]:  # Nur erste 100 Produkte
        prompt = generate_prompt_from_group(group)
        row_result = {"ProductId": product_id, "Prompt": prompt}

        for name, model_id in MODELS.items():
            try:
                print(f"⏳ Anfrage an {name} für Produkt {product_id}...")
                output = run_prompt(model_id, prompt)
                row_result[f"output_{name}"] = output
                time.sleep(1.5)  # Vermeidung von Rate Limits
            except Exception as e:
                row_result[f"output_{name}"] = f"Fehler: {str(e)}"

        results.append(row_result)
        # 💾 Speichern
    pd.DataFrame(results).to_csv("llm_vergleich_resultate_100_1.csv", index=False)
    print("✅ Ergebnisse gespeichert unter 'llm_vergleich_resultate_100_1.csv'.")

In [8]:
if __name__ == "__main__":
    main()

⏳ Anfrage an gpt-4 für Produkt 200440...
⏳ Anfrage an claude-3-opus für Produkt 200440...
⏳ Anfrage an llama-3-70b für Produkt 200440...
⏳ Anfrage an mistral-7b für Produkt 200440...
⏳ Anfrage an gpt-4 für Produkt 203240...
⏳ Anfrage an claude-3-opus für Produkt 203240...
⏳ Anfrage an llama-3-70b für Produkt 203240...
⏳ Anfrage an mistral-7b für Produkt 203240...
⏳ Anfrage an gpt-4 für Produkt 204460...
⏳ Anfrage an claude-3-opus für Produkt 204460...
⏳ Anfrage an llama-3-70b für Produkt 204460...
⏳ Anfrage an mistral-7b für Produkt 204460...
⏳ Anfrage an gpt-4 für Produkt 207900...
⏳ Anfrage an claude-3-opus für Produkt 207900...
⏳ Anfrage an llama-3-70b für Produkt 207900...
⏳ Anfrage an mistral-7b für Produkt 207900...
⏳ Anfrage an gpt-4 für Produkt 213020...
⏳ Anfrage an claude-3-opus für Produkt 213020...
⏳ Anfrage an llama-3-70b für Produkt 213020...
⏳ Anfrage an mistral-7b für Produkt 213020...
⏳ Anfrage an gpt-4 für Produkt 217400...
⏳ Anfrage an claude-3-opus für Produkt 21740